In [7]:
import pickle
import numpy as np
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
from nilearn import datasets
import glob
import json
from graph import cal_graph
from statsmodels.stats import nonparametric

In [2]:
atlas_labels = datasets.fetch_atlas_aal()["labels"]

In [8]:
with open("datasets.json", "r") as f:
    datasets = json.load(f)

In [13]:
# 参数检验
def stats_tests(preop, postop):
    # 正态性检验
    norm_stat1, p_value_norm1 = stats.shapiro(preop)
    print("正态性检验preop: ", p_value_norm1)
    norm_stat2, p_value_norm2 = stats.shapiro(postop)
    print("正态性检验postop: ", p_value_norm2)
    l_stat, p_value_l = stats.levene(preop, postop)
    print("方差齐性检验: ", p_value_l)
    # student T 检验
    t_stat, p_value = stats.ttest_ind(preop, postop)
    print("t检验: ", t_stat, p_value)
    # ANOVA
    f_stat, p_value_f = stats.f_oneway(preop, postop)
    print("ANOVA检验: ", f_stat, p_value_f)
    # 非参数检验
    # 不要求正态分布
    print("非参数检验: ")
    # Wilcoxon秩和检验
    rs_stat, p_value_rs = stats.ranksums(preop, postop)
    print("Wilcoxon秩和检验: ", rs_stat, p_value_rs)
    # 曼-惠特尼U检验
    mw_stat, p_value_m = stats.mannwhitneyu(preop, postop)
    print("曼-惠特尼U检验: ", mw_stat, p_value_m)
    # ks检验
    ks_stat, p_value_ks = stats.ks_2samp(preop, postop)
    print("ks检验: ", ks_stat, p_value_ks)
    # kruskal 原假设是各样本服从的概率分布具有相同的中位数，原假设被拒绝意味着至少一个样本的概率分布的中位数不同于其他样本
    k_stat, p_value_k = stats.kruskal(preop, postop)
    print("kruskal检验", k_stat, p_value_k)
    # Dunnett's t 检验
    # SNK q检验

In [3]:
with open("time_series2.pkl", "rb") as f:
    time_series = pickle.load(f)

In [22]:
cnt1 = 0
cnt2 = 0
for i in range(116):
    ts_preop = []
    ts_postop = []
    for sub in time_series:
        for run in time_series[sub]["ses-preop"]:
            ts_preop.append(time_series[sub]["ses-preop"][run]["time_series"].T[i])
    for sub in time_series:
        for run in time_series[sub]["ses-postop"]:
            ts_postop.append(time_series[sub]["ses-postop"][run]["time_series"].T[i])
    # time_series["sub-292"]["ses-preop"]["run-02"]["time_series"].T[i]
    # time_series["sub-292"]["ses-postop"]["run-01"]["time_series"].T[i]
    ts_preop = np.hstack(ts_preop)
    ts_postop = np.hstack(ts_postop)

    # Wilcoxon秩和检验
    rs_stat, p_value_rs = stats.ranksums(ts_preop, ts_postop)
    # print("Wilcoxon秩和检验: ", rs_stat, p_value_rs)
    # 曼-惠特尼U检验
    mw_stat, p_value_m = stats.mannwhitneyu(ts_preop, ts_postop)
    # print("曼-惠特尼U检验: ", mw_stat, p_value_m)
    if p_value_rs < 0.01 and p_value_m < 0.01:
        print(atlas_labels[i], "存在明显差异")
        cnt1 += 1
    elif p_value_rs >= 0.01 and p_value_m >= 0.01:
        # print(atlas_labels[i], "不存在明显差异")
        cnt2 += 1
    else:
        print(atlas_labels[i], "无法确定")
print(cnt1, cnt2)

Frontal_Med_Orb_R 存在明显差异
Rectus_L 存在明显差异
Vermis_6 存在明显差异
3 113


In [20]:
with open("FCs.pkl", "rb") as f:
    FCs = pickle.load(f)

In [22]:
graphs = {}
threshold = 50
for subid in FCs:
    if len(FCs[subid]["ses-preop"]) == 0:
        continue
    graphs[subid] = {
        "ses-preop":{},
        "ses-postop":{}
    }
    for run, fc in FCs[subid]["ses-preop"].items():
        graphs[subid]["ses-preop"][run] = cal_graph(fc, threshold)
    for run, fc in FCs[subid]["ses-postop"].items():
        graphs[subid]["ses-postop"][run] = cal_graph(fc, threshold)
with open("graph_theory/graphs_static.pkl", "wb") as f:
    pickle.dump(graphs, f)

In [3]:
with open("graph_theory/graphs.pkl", "rb") as f:
    graphs = pickle.load(f)

In [4]:
core_num_preop = []
core_num_postop = []

degree_centrality_preop = []
degree_centrality_postop = []
closeness_centrality_preop = []
closeness_centrality_postop = []
betweenness_centrality_preop = []
betweenness_centrality_postop = []
eigenvector_centrality_preop = []
eigenvector_centrality_postop = []

triangles_preop = []
triangles_postop = []
clustering_preop = []
clustering_postop = []
transitivity_preop = []
transitivity_postop = []

degree_assortativity_coefficient_preop = []
degree_assortativity_coefficient_postop = []
average_neighbor_degree_preop = []
average_neighbor_degree_postop = []
# average_degree_connectivity_preop = []
# average_degree_connectivity_postop = []

global_efficiency_preop = []
global_efficiency_postop = []
local_efficiency_preop = []
local_efficiency_postop = []
ratio_preop = []
ratio_postop = []

for subid, graph in graphs.items():
    for run in graph["ses-preop"]:
        core_num_preop.append(max(nx.core_number(graph["ses-preop"][run]["main_core"]).values()))
        
        degree_centrality_preop.append(list(graph["ses-preop"][run]["degree_centrality"].values()))
        closeness_centrality_preop.append(list(graph["ses-preop"][run]["closeness_centrality"].values()))
        betweenness_centrality_preop.append(list(graph["ses-preop"][run]["betweenness_centrality"].values()))
        if "eigenvector_centrality" in graph["ses-preop"][run]:
            eigenvector_centrality_preop.append(list(graph["ses-preop"][run]["eigenvector_centrality"].values()))
        
        triangles_preop.append(list(graph["ses-preop"][run]["triangles"].values()))
        clustering_preop.append(list(graph["ses-preop"][run]["clustering"].values()))
        transitivity_preop.append(graph["ses-preop"][run]["transitivity"])
        
        degree_assortativity_coefficient_preop.append(graph["ses-preop"][run]["degree_assortativity_coefficient"])
        average_neighbor_degree_preop.append(list(graph["ses-preop"][run]["average_neighbor_degree"].values()))
        # average_degree_connectivity_preop.append(graph["ses-preop"][run]["average_degree_connectivity"])
        
        global_efficiency_preop.append(graph["ses-preop"][run]["global_efficiency"])
        local_efficiency_preop.append(graph["ses-preop"][run]["local_efficiency"])
        ratio_preop.append(graph["ses-preop"][run]["ratio"])
    for run in graph["ses-postop"]:
        core_num_postop.append(max(nx.core_number(graph["ses-postop"][run]["main_core"]).values()))
        
        degree_centrality_postop.append(list(graph["ses-postop"][run]["degree_centrality"].values()))
        closeness_centrality_postop.append(list(graph["ses-postop"][run]["closeness_centrality"].values()))
        betweenness_centrality_postop.append(list(graph["ses-postop"][run]["betweenness_centrality"].values()))
        if "eigenvector_centrality" in graph["ses-postop"][run]:
            eigenvector_centrality_postop.append(list(graph["ses-postop"][run]["eigenvector_centrality"].values()))
        
        triangles_postop.append(list(graph["ses-postop"][run]["triangles"].values()))
        clustering_postop.append(list(graph["ses-postop"][run]["clustering"].values()))
        transitivity_postop.append(graph["ses-postop"][run]["transitivity"])
        
        degree_assortativity_coefficient_postop.append(graph["ses-postop"][run]["degree_assortativity_coefficient"])
        average_neighbor_degree_postop.append(list(graph["ses-postop"][run]["average_neighbor_degree"].values()))
        # average_degree_connectivity_postop.append(list(graph["ses-postop"][run]["average_degree_connectivity"].values()))
        
        global_efficiency_postop.append(graph["ses-postop"][run]["global_efficiency"])
        local_efficiency_postop.append(graph["ses-postop"][run]["local_efficiency"])
        ratio_postop.append(graph["ses-postop"][run]["ratio"])

In [5]:
# k core最大值
core_num_preop = np.array(core_num_preop)
core_num_postop = np.array(core_num_postop)

# 中心性，非单个数值，每个节点对应一个
degree_centrality_preop = np.array(degree_centrality_preop)
degree_centrality_postop = np.array(degree_centrality_postop)
closeness_centrality_preop = np.array(closeness_centrality_preop)
closeness_centrality_postop = np.array(closeness_centrality_postop)
betweenness_centrality_preop = np.array(betweenness_centrality_preop)
betweenness_centrality_postop = np.array(betweenness_centrality_postop)
eigenvector_centrality_preop = np.array(eigenvector_centrality_preop)
eigenvector_centrality_postop = np.array(eigenvector_centrality_postop)

# 三角形数，与聚类系数相关
triangles_preop = np.array(triangles_preop)
triangles_postop = np.array(triangles_postop)
# 聚类系数，每节点对应一个值，可取整体平均值
clustering_preop = np.array(clustering_preop)
clustering_postop = np.array(clustering_postop)
# 转移效率，单个值
transitivity_preop = np.array(transitivity_preop)
transitivity_postop = np.array(transitivity_postop)

# 网络关联性
degree_assortativity_coefficient_preop = np.array(degree_assortativity_coefficient_preop)
degree_assortativity_coefficient_postop = np.array(degree_assortativity_coefficient_postop)
average_neighbor_degree_preop = np.array(average_neighbor_degree_preop)
average_neighbor_degree_postop = np.array(average_neighbor_degree_postop)
# average_degree_connectivity_preop = np.array(average_degree_connectivity_preop)
# average_degree_connectivity_postop = np.array(average_degree_connectivity_postop)

# 全局与局部效率
global_efficiency_preop = np.array(global_efficiency_preop)
global_efficiency_postop = np.array(global_efficiency_postop)
local_efficiency_preop = np.array(local_efficiency_preop)
local_efficiency_postop = np.array(local_efficiency_postop)
ratio_preop = np.array(ratio_preop)
ratio_postop = np.array(ratio_postop)

In [9]:
runs = {}
for sub in graphs:
    if len(graphs[sub]["ses-preop"].keys()) == 0:
           print(sub)
    else:
        runs[sub] = [len(graphs[sub]["ses-preop"].keys()), len(graphs[sub]["ses-postop"].keys())]

sub-339
sub-357
sub-369
sub-394
sub-395
sub-403


In [11]:
Cpre =clustering_preop.sum(axis=1)/116
Cpost = clustering_postop.sum(axis=1)/116

In [12]:
lastpre = 0
lastpost = 0
cnt1 = 0
cnt2 = 0
for sub, val in runs.items():
    pre, post = val
    # print(lastpre+pre, lastpost+post)
    if nonparametric.rank_compare_2indep(Cpre[lastpre:lastpre+pre], Cpost[lastpost:lastpost+post])[0] > 0:
        cnt1 += 1
        print(f"{sub}电刺激后聚类系数降低: {datasets[sub]['soz']}")
    else:
        cnt2 += 1
        print(f"{sub}电刺激后聚类系数增加: {datasets[sub]['soz']}")
    lastpre += pre
    lastpost += post
print(cnt1, cnt2)

sub-292电刺激后聚类系数降低: 左内侧颞叶、左额叶
sub-294电刺激后聚类系数增加: 右前额叶
sub-302电刺激后聚类系数增加: 左内侧颞叶
sub-303电刺激后聚类系数增加: 右内侧颞叶，左内侧颞叶脑软化
sub-307电刺激后聚类系数增加: 左侧脑岛
sub-314电刺激后聚类系数降低: 双侧内侧颞叶
sub-316电刺激后聚类系数增加: 右内侧颞叶
sub-320电刺激后聚类系数增加: 右侧海马体
sub-330电刺激后聚类系数增加: 左枕叶
sub-331电刺激后聚类系数增加: 左内侧颞叶
sub-334电刺激后聚类系数增加: 右颞极，左颞基底
sub-335电刺激后聚类系数降低: 双侧内侧颞叶
sub-352电刺激后聚类系数增加: 左额囊性瘤
sub-372电刺激后聚类系数降低: 左颞极
sub-376电刺激后聚类系数增加: 右内侧颞叶
sub-384电刺激后聚类系数增加: 右内侧颞叶，右额叶
sub-399电刺激后聚类系数增加: 右内侧颞叶，可能是右额基
sub-400电刺激后聚类系数增加: 左内侧颞叶
sub-405电刺激后聚类系数降低: 左额脑软化症
sub-413电刺激后聚类系数降低: 右内侧颞叶
6 14


c:\Users\fjj\anaconda3\lib\site-packages\statsmodels\stats\nonparametric.py:473: RuntimeWarning: invalid value encountered in double_scalars
  S1 /= nobs1 - 1
c:\Users\fjj\anaconda3\lib\site-packages\statsmodels\stats\nonparametric.py:478: RuntimeWarning: divide by zero encountered in double_scalars
  wbfn /= (nobs1 + nobs2) * np.sqrt(nobs1 * S1 + nobs2 * S2)
c:\Users\fjj\anaconda3\lib\site-packages\statsmodels\stats\nonparametric.py:485: RuntimeWarning: invalid value encountered in double_scalars
  df = df_numer / df_denom


In [33]:
print(Cpre.sum()/Cpre.shape[0], Cpost.sum()/Cpost.shape[0])

0.708467793831942 0.7475613831096029


In [14]:
stats_tests(Cpre, Cpost)

正态性检验preop:  0.01409502699971199
正态性检验postop:  0.1541590392589569
方差齐性检验:  0.557807994536543
-2.600465448680086 0.009902495762993478
6.762420549778919 0.009902495762993516
非参数检验: 
Wilcoxon秩和检验:  -2.2031185552528263 0.0275863937043405
曼-惠特尼U检验:  4922.0 0.02765888241623276
ks检验:  0.18051384718051386 0.06265342997121903
kruskal检验 4.853731368499211 0.027586393704341922


In [59]:
nonparametric.rank_compare_2indep(Cpre, Cpost)

<class 'statsmodels.stats.nonparametric.RankCompareResult'>
statistic = -2.231985150235751
pvalue = 0.027222118950005723
s1 = 2171.760088855979
s2 = 431.90230810520677
var1 = 0.08275263255814583
var2 = 0.07887186050131606
var = 0.37881354852836296
var_prob = 0.001605142154781199
nobs1 = 74
nobs2 = 162
nobs = 236
mean1 = 104.01351351351352
mean2 = 125.11728395061728
prob1 = 0.41057724391057726
prob2 = 0.5894227560894226
somersd1 = -0.17884551217884548
somersd2 = 0.17884551217884526
df = 138.49771236236373
use_t = True
tuple = (-2.231985150235751, 0.027222118950005723)

In [33]:
stats_tests(global_efficiency_preop, global_efficiency_postop)

正态性检验preop:  0.08084501326084137
正态性检验postop:  0.11492002755403519
方差齐性检验:  0.7760637035969065
-1.6185829265134002 0.10688406820378152
2.6198106900006892 0.10688406820378263
非参数检验: 
Wilcoxon秩和检验:  -1.360507913784861 0.17366925029686442
曼-惠特尼U检验:  5332.0 0.17399442825298506
ks检验:  0.1284617951284618 0.33555700589828075
kruskal检验 1.8509817834712976 0.1736692502968533


In [35]:
stats_tests(local_efficiency_preop, local_efficiency_postop)

正态性检验preop:  4.257240561855724e-06
正态性检验postop:  9.959038834495004e-06
方差齐性检验:  0.26674605779545213
-2.1157201110960333 0.03542552556972661
4.476271588496191 0.03542552556972645
非参数检验: 
Wilcoxon秩和检验:  -1.755096311740591 0.07924283510101604
曼-惠特尼U检验:  5140.0 0.07941872966341001
ks检验:  0.147313980647314 0.19552345490624023
kruskal检验 3.0803630634854926 0.07924283510101274


In [45]:
stats_tests(ratio_preop, ratio_postop)

正态性检验preop:  0.000318704143865034
正态性检验postop:  4.2852956539718434e-05
方差齐性检验:  0.7515467737981812
0.907853451859428 0.36488965301061715
0.8241978900530632 0.3648896530106265
非参数检验: 
Wilcoxon秩和检验:  0.850831233092043 0.39486310904949495
曼-惠特尼U检验:  6408.0 0.3954342549377542
ks检验:  0.1284617951284618 0.33555700589828075
kruskal检验 0.7239137872049923 0.39486310904947364


In [40]:
stats_tests(core_num_preop, core_num_postop)

正态性检验preop:  0.02152283675968647
正态性检验postop:  5.073248757980764e-06
方差齐性检验:  0.1040737145494205
-2.0948753888578295 0.03725770726675177
4.388502894842248 0.03725770726675233
非参数检验: 
Wilcoxon秩和检验:  -1.6420631769095226 0.1005769119221547
曼-惠特尼U检验:  5195.0 0.10070901136050447
ks检验:  0.16049382716049382 0.127845149055132
kruskal检验 2.6976546260080756 0.10049598431113478


In [41]:
stats_tests(transitivity_preop, transitivity_postop)

正态性检验preop:  0.028943326324224472
正态性检验postop:  0.03597412630915642
方差齐性检验:  0.7919193607016528
-2.424711813756636 0.016078984928837456
5.879227379771024 0.01607898492883734
非参数检验: 
Wilcoxon秩和检验:  -2.003769208368942 0.04509478869331483
曼-惠特尼U检验:  5019.0 0.04520502745295591
ks检验:  0.1558224891558225 0.1492862419558123
kruskal检验 4.015091040407469 0.0450947886933157


In [44]:
cnt1 = 0
cnt2 = 0
for sub in graphs:
    if len(graphs[sub]["ses-preop"]) == 0:
        continue
    preop = []
    for run, graph in graphs[sub]["ses-preop"].items():
        preop.append(graph["transitivity"])
    postop= []
    for run, graph in graphs[sub]["ses-postop"].items():
        postop.append(graph["transitivity"])
    if nonparametric.rank_compare_2indep(preop, postop)[0] < 0:
        cnt1 += 1
        print(sub, '增加', datasets[sub]["soz"])
    else:
        cnt2 += 1
        print(sub, '减小', datasets[sub]["soz"])
print(cnt1, cnt2)

sub-292 增加 左内侧颞叶、左额叶
sub-294 减小 右前额叶
sub-302 减小 左内侧颞叶
sub-303 增加 右内侧颞叶，左内侧颞叶脑软化
sub-307 增加 左侧脑岛
sub-314 减小 双侧内侧颞叶
sub-316 增加 右内侧颞叶
sub-320 增加 右侧海马体
sub-330 增加 左枕叶
sub-331 增加 左内侧颞叶
sub-334 增加 右颞极，左颞基底
sub-335 增加 双侧内侧颞叶
sub-352 增加 左额囊性瘤
sub-372 减小 左颞极
sub-376 增加 右内侧颞叶
sub-384 减小 右内侧颞叶，右额叶
sub-399 增加 右内侧颞叶，可能是右额基
sub-400 减小 左内侧颞叶
sub-405 增加 左额脑软化症
sub-413 减小 右内侧颞叶
13 7


c:\Users\fjj\anaconda3\lib\site-packages\statsmodels\stats\nonparametric.py:473: RuntimeWarning: invalid value encountered in double_scalars
  S1 /= nobs1 - 1


In [43]:
nonparametric.rank_compare_2indep(transitivity_preop, transitivity_postop)

<class 'statsmodels.stats.nonparametric.RankCompareResult'>
statistic = -2.061957634689352
pvalue = 0.040915080723523886
s1 = 1971.2700851536467
s2 = 479.72015182884746
var1 = 0.07511317196896992
var2 = 0.08760411830329574
var = 0.36717093032752873
var_prob = 0.0015558090268115625
nobs1 = 74
nobs2 = 162
nobs = 236
mean1 = 105.32432432432432
mean2 = 124.51851851851852
prob1 = 0.41866866866866864
prob2 = 0.5813313313313313
somersd1 = -0.16266266266266272
somersd2 = 0.1626626626626626
df = 151.94685448111537
use_t = True
tuple = (-2.061957634689352, 0.040915080723523886)

In [46]:
stats_tests(degree_assortativity_coefficient_preop, degree_assortativity_coefficient_postop)

正态性检验preop:  0.2456342577934265
正态性检验postop:  0.0007478159968741238
方差齐性检验:  0.01419746254654127
0.8700653784234633 0.385156315900146
0.757013762731164 0.3851563159001494
非参数检验: 
Wilcoxon秩和检验:  1.2474747789537926 0.21222346239551593
曼-惠特尼U检验:  6601.0 0.21260026020587064
ks检验:  0.17150483817150483 0.08726063817355101
kruskal检验 1.556193324125843 0.212223462395508
